# Model A: Time-Frequency Domain U-Net for Music Source Separation

This notebook demonstrates music source separation using a U-Net architecture in the time-frequency domain.

## 1. Imports and Environment Setup

- Import required libraries (torch, numpy, matplotlib, etc.)

- Set device (CPU/GPU)

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import torch
import numpy as np
from pathlib import Path
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

sys.path.append('..')

from models import utils
from models import model_A as ma

# Setup Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

## 2. Data Loading and Preprocessing

- Load example mixture and target data (from .npy files)

- Visualize waveforms and spectrograms

- Normalize or preprocess as needed

In [ ]:
# Force plots to appear inside the notebook
%matplotlib inline 

import sys
import numpy as np
import musdb
import matplotlib.pyplot as plt
from pathlib import Path

# Add parent directory
sys.path.append('..') 
from models import utils
from models.utils import AudioProcessor

# ==============================================================================
# 1. PREPARE CURRICULUM CACHE
# ==============================================================================
print("⏳ Checking Data Cache...")
mus = musdb.DB(download=True)
utils.prepare_curriculum_cache(mus, cache_dir="../data/curriculum")

# ==============================================================================
# 2. LOAD DATA PATHS FOR BOTH STAGES
# ==============================================================================
data_root = Path("../data/curriculum")
s1_mix_path = data_root / "stage1" / "mixture"
s1_tgt_path = data_root / "stage1" / "target"
s2_mix_path = data_root / "stage2" / "mixture"
s2_tgt_path = data_root / "stage2" / "target"

mix_files_stage1 = sorted(list(s1_mix_path.glob("*.npy")))
tgt_files_stage1 = sorted(list(s1_tgt_path.glob("*.npy")))
mix_files_stage2 = sorted(list(s2_mix_path.glob("*.npy")))
tgt_files_stage2 = sorted(list(s2_tgt_path.glob("*.npy")))

print(f"\n✅ Data Ready!")
print(f"   Stage 1 Samples: {len(mix_files_stage1)}")
print(f"   Stage 2 Samples: {len(mix_files_stage2)}")

## 3. Model Architecture

- Show the U-Net model code (import or define)

- Print model summary

In [ ]:
# 3. Model Architecture: Frequency Domain U-Net
from models import model_A as ma

# Instantiate the U-Net model
model = ma.TimeFrequencyDomainUNet(in_channels=1, out_channels=1, base_filters=64, num_layers=4).to(device)

# Print model summary
print(model)

# Optionally visualize model architecture if utility exists
if hasattr(utils, 'visualize_model_architecture'):
    utils.visualize_model_architecture(model)

## 4. Training Setup

- Define loss function and optimizer

- Set training hyperparameters (epochs, batch size, learning rate)


In [ ]:
# 4. Training Setup
import torch.nn as nn
import torch.optim as optim

# Define loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Set training hyperparameters
batch_size = 4
num_epochs = 20
patience = 5

print(f"Loss function: {loss_fn}")
print(f"Optimizer: {optimizer}")
print(f"Batch size: {batch_size}, Epochs: {num_epochs}, Patience: {patience}")

## 5. Training Loop

### 5.a Overfit Method

- Train the model on a very small subset (e.g., one batch or a few samples) to ensure it can overfit and the implementation is correct.


In [ ]:
# 5.a Overfit Method: Try to overfit on a tiny subset of Stage 1 data
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from pathlib import Path
import os

# Force plots to display inline
%matplotlib inline 

from models import utils
from models import model_A as ma

# 1. Prepare Data
# ----------------------------------------------------------------------------
stage1_mix_path = Path("../data/curriculum/stage1/mixture")
stage1_tgt_path = Path("../data/curriculum/stage1/target")
mix_files = sorted(list(stage1_mix_path.glob("*.npy")))
tgt_files = sorted(list(stage1_tgt_path.glob("*.npy")))
# ----------------------------------------------------------------------------

overfit_idx = 0
sr = 22050
device = 'cuda' if torch.cuda.is_available() else 'cpu'

checkpoint_dir = "../checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "overfit_checkpoint.pth")

if len(mix_files) > 0 and len(tgt_files) > 0:
    # Load raw numpy arrays (2 seconds)
    mix_wav = np.load(mix_files[overfit_idx])[:sr*2] 
    tgt_wav = np.load(tgt_files[overfit_idx])[:sr*2]

    # Initialize Processor (needed for Trainer)
    processor = utils.AudioProcessor(device=device)

    # 2. Create Tiny Dataset (Pass RAW AUDIO, not spectrograms)
    class TinyDataset(torch.utils.data.Dataset):
        def __init__(self, mix, tgt):
            self.mix = mix
            self.tgt = tgt
        def __len__(self):
            return 1
        def __getitem__(self, idx):
            # FIXED: Return keys 'mix' and 'tgt' to match ModelATrainer expectations
            return {
                'mix': torch.tensor(self.mix, dtype=torch.float32), 
                'tgt': torch.tensor(self.tgt, dtype=torch.float32)
            }

    tiny_dataset = TinyDataset(mix_wav, tgt_wav)
    tiny_loader = DataLoader(tiny_dataset, batch_size=1, shuffle=False)

    # 3. Initialize Model & Trainer
    model = ma.TimeFrequencyDomainUNet(in_channels=1, out_channels=1, base_filters=32, num_layers=4).to(device)
    
    # Large patience effectively disables early stopping for this test
    trainer = ma.ModelATrainer(model, tiny_loader, tiny_loader, processor, learning_rate=1e-3, device=device, patience=9999)

    if os.path.exists(checkpoint_path):
        print(f"Loading overfit checkpoint from {checkpoint_path}")
        # Note: If you want to FORCE retraining, delete the checkpoint file manually or uncomment the line below:
        # os.remove(checkpoint_path)
        model.load_state_dict(torch.load(checkpoint_path, map_location=device))
        print("Skipping training (checkpoint loaded).")
    else:
        print("🚀 Starting Overfit Test...")
        # Capture the history dictionary returned by train()
        history = trainer.train(num_epochs=50) # Increased to 50 for better overfit proof
        
        # Save manually if needed, though trainer usually saves best
        torch.save(model.state_dict(), checkpoint_path)

        # 4. Plot Loss
        if len(history['train_loss']) > 0:
            plt.figure(figsize=(8, 4))
            plt.plot(history['train_loss'], label='Train Loss', marker='o')
            plt.plot(history['val_loss'], label='Val Loss', marker='x')
            plt.xlabel('Epoch')
            plt.ylabel('Loss (L1)')
            plt.title('Overfit Training Loss Curve')
            plt.legend()
            plt.grid(True)
            plt.show()

else:
    print("⚠️ No data found. Please run the caching step first.")

### 5.b Full Training

- Train the model on the full training dataset as intended.


In [ ]:
# 5.b Full Training: Train on Stage 1, then Stage 2
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from pathlib import Path
import os

# Force plots to be inline
%matplotlib inline 

from models import utils
from models import model_A as ma

# ==============================================================================
# CONFIGURATION
# ==============================================================================
batch_size = 4
num_epochs = 20
learning_rate = 1e-4
patience = 5
device = 'cuda' if torch.cuda.is_available() else 'cpu'
checkpoint_dir = "../checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# 1. Setup Processor & Model
# ------------------------------------------------------------------------------
processor = utils.AudioProcessor(device=device)
model = ma.TimeFrequencyDomainUNet(in_channels=1, out_channels=1, base_filters=64, num_layers=4).to(device)

# ==============================================================================
# STAGE 1 TRAINING (Vocals + Other -> Other)
# ==============================================================================
print("\n" + "="*50)
print("🚀 STAGE 1: Training on Simplified Mixture")
print("="*50)

# 1. Prepare Paths
s1_root = Path("../data/curriculum/stage1")
s1_mix_files = sorted(list((s1_root / "mixture").glob("*.npy")))
s1_tgt_files = sorted(list((s1_root / "target").glob("*.npy")))

# 2. Create Datasets (Using utils.SpectrogramDataset which loads raw waves)
split_idx = int(len(s1_mix_files) * 0.8)
train_ds_s1 = utils.SpectrogramDataset(s1_mix_files[:split_idx], s1_tgt_files[:split_idx])
val_ds_s1 = utils.SpectrogramDataset(s1_mix_files[split_idx:], s1_tgt_files[split_idx:])

train_loader_s1 = DataLoader(train_ds_s1, batch_size=batch_size, shuffle=True)
val_loader_s1 = DataLoader(val_ds_s1, batch_size=batch_size, shuffle=False)

# 3. Check for Checkpoint
stage1_ckpt = os.path.join(checkpoint_dir, "model_a_stage1.pth")

if os.path.exists(stage1_ckpt):
    print(f"✓ Found Stage 1 checkpoint at {stage1_ckpt}. Loading...")
    model.load_state_dict(torch.load(stage1_ckpt, map_location=device))
else:
    # 4. Train
    # We initialize the trainer. It handles the epoch loop and printing internally.
    trainer_s1 = ma.ModelATrainer(
        model, 
        train_loader_s1, 
        val_loader_s1, 
        processor, 
        learning_rate=learning_rate, 
        device=device, 
        patience=patience
    )
    
    # This call will print progress epoch-by-epoch as they finish
    history_s1 = trainer_s1.train(num_epochs=num_epochs)
    
    # Save Checkpoint
    torch.save(model.state_dict(), stage1_ckpt)
    
    # Plot
    if len(history_s1['train_loss']) > 0:
        plt.figure(figsize=(10, 4))
        plt.plot(history_s1['train_loss'], label='Train Loss')
        plt.plot(history_s1['val_loss'], label='Val Loss')
        plt.title("Stage 1 Training Results")
        plt.legend()
        plt.show()

# ==============================================================================
# STAGE 2 TRAINING (Full Mix -> Other)
# ==============================================================================
print("\n" + "="*50)
print("🚀 STAGE 2: Fine-Tuning on Full Mixture")
print("="*50)

# 1. Prepare Paths
s2_root = Path("../data/curriculum/stage2")
s2_mix_files = sorted(list((s2_root / "mixture").glob("*.npy")))
s2_tgt_files = sorted(list((s2_root / "target").glob("*.npy")))

# 2. Create Datasets
train_ds_s2 = utils.SpectrogramDataset(s2_mix_files[:split_idx], s2_tgt_files[:split_idx])
val_ds_s2 = utils.SpectrogramDataset(s2_mix_files[split_idx:], s2_tgt_files[split_idx:])

train_loader_s2 = DataLoader(train_ds_s2, batch_size=batch_size, shuffle=True)
val_loader_s2 = DataLoader(val_ds_s2, batch_size=batch_size, shuffle=False)

# 3. Check for Checkpoint
stage2_ckpt = os.path.join(checkpoint_dir, "model_a_stage2.pth")

if os.path.exists(stage2_ckpt):
    print(f"✓ Found Stage 2 checkpoint at {stage2_ckpt}. Loading...")
    model.load_state_dict(torch.load(stage2_ckpt, map_location=device))
else:
    # 4. Train (Fine-tune with lower learning rate)
    trainer_s2 = ma.ModelATrainer(
        model, 
        train_loader_s2, 
        val_loader_s2, 
        processor, 
        learning_rate=learning_rate * 0.1,  # Lower LR for fine-tuning
        device=device, 
        patience=patience
    )
    
    history_s2 = trainer_s2.train(num_epochs=num_epochs)
    
    # Save Checkpoint
    torch.save(model.state_dict(), stage2_ckpt)
    
    # Plot
    if len(history_s2['train_loss']) > 0:
        plt.figure(figsize=(10, 4))
        plt.plot(history_s2['train_loss'], label='Train Loss')
        plt.plot(history_s2['val_loss'], label='Val Loss')
        plt.title("Stage 2 Training Results")
        plt.legend()
        plt.show()

## 6. Evaluation and Inference

- Run the trained model on test data

- Visualize separated sources (waveforms, spectrograms)

- Optionally, compute evaluation metrics (e.g., SDR, SIR)


In [ ]:
# Plot loss curves from a .pt checkpoint file
import torch
import matplotlib.pyplot as plt

# Path to your checkpoint (update as needed)
ckpt_path = "../checkpoints/model_a_stage1.pth"  # or model_a_stage2.pth

ckpt = torch.load(ckpt_path, map_location='cpu')
if 'history' in ckpt:
    history = ckpt['history']
    plt.figure(figsize=(10, 4))
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.title("Loss Curves from Checkpoint")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()
else:
    print("No loss history found in checkpoint.")

## 7. Discussion and Next Steps

- Summarize results

- Suggest improvements or further experiments